## Clean Data

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

# Notebook is in: .../iaqf_analysis/stablecoin_dynamics/code
NOTEBOOK_DIR = Path.cwd()
ROOT = NOTEBOOK_DIR.parents[1]

DATA_KRAKEN_RAW = ROOT / "data" / "kraken" / "raw"
print("notebook_dir:", NOTEBOOK_DIR)
print("root:", ROOT)
print("data_kraken_raw:", DATA_KRAKEN_RAW)
print("exists:", DATA_KRAKEN_RAW.exists())
print("csv files:", [p.name for p in sorted(DATA_KRAKEN_RAW.glob("*.csv"))])

notebook_dir: /Users/zhaotanji/Desktop/iaqf_analysis/stablecoin_dynamics/code
root: /Users/zhaotanji/Desktop/iaqf_analysis
data_kraken_raw: /Users/zhaotanji/Desktop/iaqf_analysis/data/kraken/raw
exists: True
csv files: ['BTCUSDC_1m_20230301_20230321.csv', 'BTCUSDT_1m_20230301_20230321.csv', 'BTCUSD_1m_20230301_20230321.csv']


In [2]:
# Update filenames here if your repo changes them
files_k = {
    "btc_usdt": DATA_KRAKEN_RAW / "BTCUSDT_1m_20230301_20230321.csv",
    "btc_usdc": DATA_KRAKEN_RAW / "BTCUSDC_1m_20230301_20230321.csv",
    "btc_usd":  DATA_KRAKEN_RAW / "BTCUSD_1m_20230301_20230321.csv",
}

for k, p in files_k.items():
    print(k, "->", p, "| exists:", p.exists())

data_k = {k: pd.read_csv(p, header=None) for k, p in files_k.items()}
for k, df in data_k.items():
    print(k, df.shape)
    display(df.head(2))

btc_usdt -> /Users/zhaotanji/Desktop/iaqf_analysis/data/kraken/raw/BTCUSDT_1m_20230301_20230321.csv | exists: True
btc_usdc -> /Users/zhaotanji/Desktop/iaqf_analysis/data/kraken/raw/BTCUSDC_1m_20230301_20230321.csv | exists: True
btc_usd -> /Users/zhaotanji/Desktop/iaqf_analysis/data/kraken/raw/BTCUSD_1m_20230301_20230321.csv | exists: True
btc_usdt (79372, 7)


,0,1,2,3,4,5,6
0,1672531260,16540.3,16540.3,16540.3,16540.3,0.003474,1
1,1672531320,16540.3,16540.3,16540.3,16540.3,0.001209,1


btc_usdc (56717, 7)


,0,1,2,3,4,5,6
0,1672531200,16535.97,16535.97,16522.66,16522.66,0.014299,2
1,1672531740,16513.41,16513.41,16513.41,16513.41,0.000396,1


btc_usd (129298, 7)


,0,1,2,3,4,5,6
0,1672531200,16528.7,16530.0,16528.6,16529.9,0.984157,30
1,1672531260,16529.9,16530.0,16527.0,16527.0,0.266923,6


In [3]:
import pandas as pd
import numpy as np

def parse_time_column(s: pd.Series) -> pd.Series:
    """
    Robust timestamp parser:
    - numeric epoch seconds OR milliseconds
    - or datetime-like strings
    """
    s_num = pd.to_numeric(s, errors="coerce")
    numeric_share = s_num.notna().mean()

    if numeric_share > 0.8:
        med = float(np.nanmedian(s_num.values))
        # epoch ms ~ 1e12, epoch s ~ 1e9
        unit = "ms" if med > 1e11 else "s"
        ts = pd.to_datetime(s_num, unit=unit, utc=True)
    else:
        ts = pd.to_datetime(s, utc=True, errors="coerce")

    return ts

def prepare_price_series_any(df: pd.DataFrame, label: str) -> pd.Series:
    """
    Returns a close-price series indexed by UTC timestamp.
    Works for:
    - Binance format: columns include open_time + close
    - Kraken no-header format: col0=timestamp, col4=close
    """
    out = df.copy()

    # Case 1: has named columns (Binance-like)
    if "close" in out.columns:
        time_col_candidates = ["open_time", "timestamp", "time", "openTime"]
        time_col = next((c for c in time_col_candidates if c in out.columns), None)
        if time_col is None:
            raise ValueError(f"[{label}] No timestamp column found. Columns={list(out.columns)}")

        out["timestamp"] = parse_time_column(out[time_col])
        out["close"] = pd.to_numeric(out["close"], errors="coerce")

    # Case 2: Kraken no-header numeric columns
    else:
        # expect: 0=timestamp, 4=close
        if out.shape[1] < 5:
            raise ValueError(f"[{label}] Kraken raw expected >=5 cols, got {out.shape[1]}")
        out = out.rename(columns={0: "timestamp_raw", 4: "close_raw"})
        out["timestamp"] = parse_time_column(out["timestamp_raw"])
        out["close"] = pd.to_numeric(out["close_raw"], errors="coerce")

    out = out.dropna(subset=["timestamp", "close"])
    out = out[["timestamp", "close"]].sort_values("timestamp")
    out = out.drop_duplicates(subset=["timestamp"], keep="last")
    return out.set_index("timestamp")["close"].rename(label)

# Build close series
p_btc_usdt_k = prepare_price_series_any(data_k["btc_usdt"], "p_btc_usdt_k")
p_btc_usdc_k = prepare_price_series_any(data_k["btc_usdc"], "p_btc_usdc_k")
p_btc_usd_k  = prepare_price_series_any(data_k["btc_usd"],  "p_btc_usd_k")

prices_k = pd.concat([p_btc_usdt_k, p_btc_usdc_k, p_btc_usd_k], axis=1)
print("Rows total (union):", len(prices_k))
print("Missing counts:\n", prices_k.isna().sum())

prices_k_clean = prices_k.dropna().copy()
print("Rows (intersection):", len(prices_k_clean))
print("Clean panel range:", prices_k_clean.index.min(), "to", prices_k_clean.index.max())
print("Clean panel shape:", prices_k_clean.shape)


Rows total (union): 129383
Missing counts:
 p_btc_usdt_k    50011
p_btc_usdc_k    72666
p_btc_usd_k        85
dtype: int64
Rows (intersection): 38942
Clean panel range: 2023-01-01 00:09:00+00:00 to 2023-03-31 23:59:00+00:00
Clean panel shape: (38942, 3)


Robust timestamp parser + cleaner

## 1.1 Stablecoin Basis (Binance, 1-Minute Data)

### Objective
This section measures whether stablecoin-quoted BTC spot markets trade at a **premium or discount** relative to the **USD-quoted** BTC spot market on the **same exchange (Binance)**.

---

### Data preparation (what we did)
We load 1-minute spot **close** prices from Binance for three markets:

- $P_{\mathrm{BTC/USDT},b}(t)$ : BTC price quoted in USDT on Binance  
- $P_{\mathrm{BTC/USDC},b}(t)$ : BTC price quoted in USDC on Binance  
- $P_{\mathrm{BTC/USD},b}(t)$  : BTC price quoted in USD on Binance  

Then we:

1. Parse timestamps into a unified **UTC** datetime index
2. Sort and deduplicate timestamps
3. Align the three series on the **common intersection of timestamps** (drop any minute where one market is missing)

After cleaning, we work with a **balanced panel** where all three prices exist at each timestamp.

---

### Definition: Stablecoin basis (Binance)

**USDT basis**
$$
\mathrm{Basis}_{\mathrm{USDT},b}(t)
=
\frac{P_{\mathrm{BTC/USDT},b}(t)-P_{\mathrm{BTC/USD},b}(t)}
     {P_{\mathrm{BTC/USD},b}(t)}
$$

**USDC basis**
$$
\mathrm{Basis}_{\mathrm{USDC},b}(t)
=
\frac{P_{\mathrm{BTC/USDC},b}(t)-P_{\mathrm{BTC/USD},b}(t)}
     {P_{\mathrm{BTC/USD},b}(t)}
$$

These are **unitless relative price deviations** (fractions of the USD BTC price), so they are directly comparable across time and (later) across exchanges.

---

### Interpretation (how to read the numbers)

- **Positive basis** ($>0$):  
  BTC is **more expensive** when quoted in that stablecoin than in USD.  
  Equivalent interpretation: **the stablecoin is at a relative discount vs USD** (you need *more* stablecoin per BTC), or there is stronger demand / more friction on that stablecoin rail.

- **Negative basis** ($<0$):  
  BTC is **cheaper** when quoted in that stablecoin than in USD.  
  Equivalent interpretation: **the stablecoin is at a relative premium vs USD** (you need *less* stablecoin per BTC), or USD-side frictions make USD quotes relatively higher.

- **Magnitude**:  
  Larger absolute values mean a larger deviation from parity (stronger “stress / confidence / friction” signal).

---

### Economic meaning
The stablecoin basis captures how much the market is willing to pay (or accept) for BTC when settlement occurs via a **stablecoin rail** rather than **USD**. Persistent or asymmetric basis behavior may reflect:

- differences in perceived **counterparty / redemption risk**
- **liquidity frictions** between fiat and stablecoin rails
- shifts in **market confidence** during calm vs stress periods


In [5]:
p_usdt_k = prices_k_clean["p_btc_usdt_k"]
p_usdc_k = prices_k_clean["p_btc_usdc_k"]
p_usd_k  = prices_k_clean["p_btc_usd_k"]

basis_usdt_k = ((p_usdt_k - p_usd_k) / p_usd_k).rename("basis_usdt_k")
basis_usdc_k = ((p_usdc_k - p_usd_k) / p_usd_k).rename("basis_usdc_k")

print("basis_usdt_k describe:\n", basis_usdt_k.describe())
print("\nbasis_usdc_k describe:\n", basis_usdc_k.describe())
print("\nCorrelation (USDT vs USDC basis):", basis_usdt_k.corr(basis_usdc_k))


basis_usdt_k describe:
 count    38942.000000
mean        -0.001029
std          0.002180
min         -0.016680
25%         -0.001133
50%         -0.000236
75%          0.000087
max          0.003435
Name: basis_usdt_k, dtype: float64

basis_usdc_k describe:
 count    38942.000000
mean         0.002959
std          0.014056
min         -0.005575
25%         -0.000205
50%          0.000120
75%          0.000548
max          0.140275
Name: basis_usdc_k, dtype: float64

Correlation (USDT vs USDC basis): -0.555146242503539


## 1.2 Cross-Stablecoin Spread (Relative Confidence)

### Objective
This section isolates **relative pricing / confidence** between **USDT** and **USDC** by differencing their basis series.
Because both bases use the same **BTC-USD** benchmark, the difference removes BTC-level moves and leaves a “**USDT vs USDC**” signal.

---

### Definition: Cross-stablecoin spread (Binance)

**USDT vs USDC spread**
$$
\mathrm{Spread}_{\mathrm{USDT-USDC},b}(t)
=
\mathrm{Basis}_{\mathrm{USDT},b}(t)
-
\mathrm{Basis}_{\mathrm{USDC},b}(t)
$$

This spread is a **unitless relative deviation** (difference of two unitless bases), so it is directly comparable across time.

---

### Interpretation (how to read the numbers)

- **Positive spread** ($>0$):  
  USDT basis is higher than USDC basis  
  $\Rightarrow$ BTC is relatively more expensive in **USDT** than in **USDC**  
  $\Rightarrow$ implies **USDT is weaker** relative to USDC (or **USDC is stronger**)

- **Negative spread** ($<0$):  
  BTC is relatively more expensive in **USDC** than in **USDT**  
  $\Rightarrow$ implies **USDC is weaker** relative to USDT (or **USDT is stronger**)

- **Magnitude**:  
  Larger absolute values mean a larger **relative divergence** in stablecoin pricing (stronger “confidence / stress / friction” signal between the two stablecoin rails).

---

### Why this is useful
- Removes BTC-level noise shared by both series  
- Focuses on **relative stablecoin pricing** rather than overall crypto moves  
- Helps interpret “which stablecoin loses the peg more?” and connect to news/regulation/confidence narratives


Compute pread + summary table

In [6]:
spread_usdt_usdc_k = (basis_usdt_k - basis_usdc_k).rename("spread_usdt_usdc_k")

measures_12_k = pd.concat([basis_usdt_k, basis_usdc_k, spread_usdt_usdc_k], axis=1)

print("1.2 measures shape:", measures_12_k.shape)
print(measures_12_k.head())
print("\nMissing values (should be 0):")
print(measures_12_k.isna().sum())

1.2 measures shape: (38942, 3)
                           basis_usdt_k  basis_usdc_k  spread_usdt_usdc_k
timestamp                                                                
2023-01-01 00:09:00+00:00      0.000714     -0.000102            0.000817
2023-01-01 00:14:00+00:00      0.000454     -0.000091            0.000545
2023-01-01 00:15:00+00:00      0.000497      0.000005            0.000491
2023-01-01 00:26:00+00:00      0.000200     -0.000151            0.000350
2023-01-01 00:33:00+00:00      0.000527     -0.000232            0.000758

Missing values (should be 0):
basis_usdt_k          0
basis_usdc_k          0
spread_usdt_usdc_k    0
dtype: int64


Frequency + tail events (and top moves)

In [8]:
# Frequency of positive vs negative spread
pos_share_k = (spread_usdt_usdc_k > 0).mean()
neg_share_k = (spread_usdt_usdc_k < 0).mean()
zero_share_k = (spread_usdt_usdc_k == 0).mean()
print(f"\nSpread > 0 share: {pos_share_k:.4f}")
print(f"Spread < 0 share: {neg_share_k:.4f}")
print(f"Spread = 0 share: {zero_share_k:.4f}")

# Tail event thresholds (absolute)
abs_spread_k = spread_usdt_usdc_k.abs()
q95_k = abs_spread_k.quantile(0.95)
q99_k = abs_spread_k.quantile(0.99)

print("\nAbsolute spread thresholds:")
print(f"95% quantile: {q95_k:.6f} ({q95_k*1e4:.2f} bps)")
print(f"99% quantile: {q99_k:.6f} ({q99_k*1e4:.2f} bps)")

# Largest events (top 10 absolute)
top_events_k = measures_12_k.assign(abs_spread_k=abs_spread_k).sort_values("abs_spread_k", ascending=False).head(10)
top_events_k



Spread > 0 share: 0.3075
Spread < 0 share: 0.6919
Spread = 0 share: 0.0006

Absolute spread thresholds:
95% quantile: 0.014236 (142.36 bps)
99% quantile: 0.099499 (994.99 bps)


,basis_usdt_k,basis_usdc_k,spread_usdt_usdc_k,abs_spread_k
timestamp,,,,
2023-03-11 07:43:00+00:00,-0.005696,0.139967,-0.145664,0.145664
2023-03-11 07:18:00+00:00,-0.005158,0.140275,-0.145433,0.145433
2023-03-11 07:48:00+00:00,-0.005618,0.139695,-0.145313,0.145313
2023-03-11 07:44:00+00:00,-0.005023,0.139336,-0.144359,0.144359
2023-03-11 07:17:00+00:00,-0.004624,0.139278,-0.143902,0.143902
2023-03-11 08:16:00+00:00,-0.004860,0.138163,-0.143023,0.143023
2023-03-11 08:17:00+00:00,-0.005285,0.136987,-0.142272,0.142272
2023-03-11 07:46:00+00:00,-0.006441,0.135677,-0.142118,0.142118
2023-03-11 07:16:00+00:00,-0.005177,0.136309,-0.141486,0.141486


## 1.3 Absolute Deviation Magnitude

### Objective
While Sections 1.1 and 1.2 focus on the **direction** of stablecoin pricing deviations (premium vs discount, relative confidence),
this section focuses on the **magnitude** of deviations **regardless of sign**.

The goal is to measure **how far prices move away from parity**, not which side they move to.

---

### Definition: Absolute basis deviation

For any basis series $\mathrm{Basis}(t)$, define its absolute deviation as:

$$
\left| \mathrm{Basis}(t) \right|
$$

This transformation removes the sign and retains only the **distance from zero (parity)**.

---

### Measures used

We summarize absolute deviations using:

- **Mean Absolute Deviation (MAD)**  
  $$
  \mathrm{MAD} = \mathbb{E}\left[ \left| \mathrm{Basis}(t) \right| \right]
  $$

- **Tail percentiles** (95th and 99th):
  $$
  Q_{0.95}\left( \left| \mathrm{Basis}(t) \right| \right), \quad
  Q_{0.99}\left( \left| \mathrm{Basis}(t) \right| \right)
  $$

These statistics capture:
- *typical deviation size* (MAD)
- *extreme stress events* (upper tail behavior)

---

### Interpretation (how to read the numbers)

- **Higher MAD**  
  → Prices deviate more from parity on average  
  → Indicates weaker arbitrage efficiency or persistent frictions

- **Higher 95th / 99th percentiles**  
  → Larger extreme deviations during stress periods  
  → Indicates vulnerability to liquidity shocks, redemptions, or market stress

- **Comparison across exchanges**  
  Because the basis is unitless and normalized by the USD price, these magnitude measures are directly comparable:
  - across stablecoins (USDT vs USDC),
  - across time,
  - and across exchanges (in later sections).

---

### Economic meaning
Absolute deviation magnitude measures **how unstable the pricing relationship is**, independent of direction.

Large absolute deviations reflect:
- breakdowns in **arbitrage**
- **liquidity fragmentation** across rails
- temporary or persistent **market stress**
- exchange-specific frictions or operational constraints

---

### What this answers
This section directly answers:

- **“Are deviations larger on some exchanges?”**
- **“Where do stablecoin pricing frictions become most severe?”**
- **“Which markets experience the strongest tail stress?”**

These magnitude measures complement directional results from Sections 1.1–1.2 and help distinguish
**frequent small noise** from **rare but severe dislocations**.


In [9]:
# --- 1.3 Absolute Deviation Magnitude (Kraken) ---
import numpy as np
import pandas as pd

# 确保1.1已经跑过
need_basis = ("basis_usdt_k" in globals()) and ("basis_usdc_k" in globals())
if not need_basis:
    raise NameError("Run 1.1 first to create basis_usdt_k / basis_usdc_k")

# 合并并对齐
basis_df_k = pd.concat([basis_usdt_k, basis_usdc_k], axis=1).dropna()

# 绝对偏离
abs_basis_k = basis_df_k.abs()
abs_basis_k.columns = ["abs_basis_usdt_k", "abs_basis_usdc_k"]

# ================================
# summary stats
# ================================
def summarize_abs(series: pd.Series) -> pd.Series:
    out = pd.Series({
        "mean_abs": series.mean(),
        "median_abs": series.median(),
        "p25": series.quantile(0.25),
        "p75": series.quantile(0.75),
        "iqr": series.quantile(0.75) - series.quantile(0.25),
        "p95": series.quantile(0.95),
        "p99": series.quantile(0.99),
        "max": series.max(),
    })
    
    # bps版本
    out_bps = out * 1e4
    out_bps.index = [f"{k}_bps" for k in out.index]
    
    return pd.concat([out, out_bps])

# ================================
# summary table
# ================================
summary_13_k = pd.concat(
    [
        summarize_abs(abs_basis_k["abs_basis_usdt_k"]).rename("USDT (Kraken)"),
        summarize_abs(abs_basis_k["abs_basis_usdc_k"]).rename("USDC (Kraken)"),
    ],
    axis=1
).T

display(summary_13_k)

# ================================
# Top 10 largest deviation minutes
# ================================
top10_usdt_k = (
    abs_basis_k["abs_basis_usdt_k"]
    .sort_values(ascending=False)
    .head(10)
    .to_frame()
)

top10_usdc_k = (
    abs_basis_k["abs_basis_usdc_k"]
    .sort_values(ascending=False)
    .head(10)
    .to_frame()
)

print("Top 10 absolute deviation timestamps (USDT Kraken):")
display(top10_usdt_k.assign(abs_bps = top10_usdt_k.iloc[:,0] * 1e4))

print("\nTop 10 absolute deviation timestamps (USDC Kraken):")
display(top10_usdc_k.assign(abs_bps = top10_usdc_k.iloc[:,0] * 1e4))


,mean_abs,median_abs,p25,p75,iqr,p95,p99,max,mean_abs_bps,median_abs_bps,p25_bps,p75_bps,iqr_bps,p95_bps,p99_bps,max_bps
USDT (Kraken),0.001233,0.000402,0.000165,0.001197,0.001031,0.005225,0.010659,0.016680,12.325987,4.018376,1.653864,11.966678,10.312814,52.254426,106.590367,166.797100
USDC (Kraken),0.003280,0.000351,0.000159,0.000745,0.000586,0.009018,0.093398,0.140275,32.797929,3.509907,1.589658,7.448662,5.859004,90.175211,933.976147,1402.747543


Top 10 absolute deviation timestamps (USDT Kraken):


,abs_basis_usdt_k,abs_bps
timestamp,,
2023-03-12 19:48:00+00:00,0.016680,166.797100
2023-03-12 19:21:00+00:00,0.016480,164.798666
2023-03-12 19:49:00+00:00,0.016454,164.538646
2023-03-12 19:19:00+00:00,0.016113,161.133870
2023-03-12 19:24:00+00:00,0.015878,158.778712
2023-03-12 19:44:00+00:00,0.015852,158.517980
2023-03-12 19:45:00+00:00,0.015819,158.191291
2023-03-12 19:29:00+00:00,0.015715,157.145961
2023-03-12 19:47:00+00:00,0.015662,156.622131



Top 10 absolute deviation timestamps (USDC Kraken):


,abs_basis_usdc_k,abs_bps
timestamp,,
2023-03-11 07:18:00+00:00,0.140275,1402.747543
2023-03-11 07:43:00+00:00,0.139967,1399.673776
2023-03-11 07:48:00+00:00,0.139695,1396.951693
2023-03-11 07:44:00+00:00,0.139336,1393.361666
2023-03-11 07:17:00+00:00,0.139278,1392.783200
2023-03-11 08:16:00+00:00,0.138163,1381.630110
2023-03-11 08:17:00+00:00,0.136987,1369.869633
2023-03-11 07:16:00+00:00,0.136309,1363.094623
2023-03-11 07:19:00+00:00,0.136102,1361.024067


## 2.1 Volatility of Basis (Rolling)

### Goal
This section measures **how unstable** the stablecoin basis is over time.
If rolling volatility spikes, it indicates periods when the stablecoin-quoted BTC price deviates from the USD-quoted BTC price in a more **erratic / stressed** way.

### Objects from Section 1
We use the basis series (unitless relative deviations):

$$
\mathrm{Basis}_{\mathrm{USDT},e}(t)=\frac{P_{\mathrm{BTC/USDT},e}(t)-P_{\mathrm{BTC/USD},e}(t)}{P_{\mathrm{BTC/USD},e}(t)}
$$

$$
\mathrm{Basis}_{\mathrm{USDC},e}(t)=\frac{P_{\mathrm{BTC/USDC},e}(t)-P_{\mathrm{BTC/USD},e}(t)}{P_{\mathrm{BTC/USD},e}(t)}
$$

### Rolling volatility definition
For a trailing window length \(k\) (minutes for 1-minute data), rolling volatility is the rolling standard deviation:

$$
\sigma_{\mathrm{basis},e}(t;k)=\mathrm{StdDev}\Big(\mathrm{Basis}_e(t-k+1),\ldots,\mathrm{Basis}_e(t)\Big)
$$

We compute this separately for USDT and USDC:

$$
\sigma_{\mathrm{USDT},e}(t;k)=\mathrm{StdDev}\Big(\mathrm{Basis}_{\mathrm{USDT},e}(t-k+1),\ldots,\mathrm{Basis}_{\mathrm{USDT},e}(t)\Big)
$$

$$
\sigma_{\mathrm{USDC},e}(t;k)=\mathrm{StdDev}\Big(\mathrm{Basis}_{\mathrm{USDC},e}(t-k+1),\ldots,\mathrm{Basis}_{\mathrm{USDC},e}(t)\Big)
$$

### Interpretation (how to read the number)
- Higher \( \sigma \)  \(\Rightarrow\) deviations are **more volatile / less stable** in that window  
- Lower \( \sigma \) \(\Rightarrow\) deviations are **more stable** (even if the level is not zero)  
- Compare **USDT vs USDC**: which stablecoin basis is more “twitchy”  
- Compare **Binance vs Kraken**: which exchange shows more unstable stablecoin pricing

### Units and scaling
Rolling volatility is still **unitless** (standard deviation of a fraction).  
For readability we report it in **basis points (bps)**:

$$
\sigma_{\mathrm{bps}} = 10{,}000 \times \sigma
$$

Example: \(0.00010 \approx 1\) bp.


In [10]:
import pandas as pd

# windows in minutes (data is 1-min)
windows = [60, 360, 1440]  # 1h, 6h, 1d

# Kraken basis from 1.1
assets_k = {
    "USDT (Kraken)": basis_usdt_k,
    "USDC (Kraken)": basis_usdc_k,
}

def summarize_vol(series: pd.Series) -> pd.Series:
    return pd.Series({
        "mean": series.mean(),
        "p25": series.quantile(0.25),
        "p75": series.quantile(0.75),
        "p90": series.quantile(0.90),
        "p99": series.quantile(0.99),
        "max": series.max(),
    })

vol_tables_k = []

for w in windows:
    rows = []
    for name, basis_k in assets_k.items():
        vol_k = basis_k.rolling(w).std()
        stats_k = summarize_vol(vol_k.dropna())
        stats_k.name = name
        rows.append(stats_k)

    table_k = pd.DataFrame(rows)
    table_k.insert(0, "window", w)
    vol_tables_k.append(table_k)

vol_summary_k = pd.concat(vol_tables_k, axis=0)

display(vol_summary_k)

,window,mean,p25,p75,p90,p99,max
USDT (Kraken),60,0.000478,0.000343,0.000565,0.000694,0.001288,0.002074
USDC (Kraken),60,0.000836,0.000405,0.000706,0.000982,0.009401,0.027529
USDT (Kraken),360,0.000592,0.000400,0.000600,0.000836,0.002595,0.003589
USDC (Kraken),360,0.001413,0.000465,0.000700,0.000973,0.023462,0.040652
USDT (Kraken),1440,0.000747,0.000423,0.000714,0.001201,0.003552,0.003631
USDC (Kraken),1440,0.002890,0.000495,0.000734,0.001309,0.044270,0.047009


## 2.2 Mean Reversion Speed (Half-Life)

### Goal
Quantify how quickly the **stablecoin basis** returns toward its typical level after a deviation.

### Model (AR(1))
We fit an AR(1) model to the basis series:

$$
\text{Basis}(t) = \alpha + \phi\,\text{Basis}(t-1) + \varepsilon_t
$$

- $\phi$ measures **persistence**:
  - If $|\phi|$ is close to 1, shocks decay slowly (sticky dislocations).
  - If $|\phi|$ is small, shocks decay quickly (fast arbitrage / fast correction).

### Half-life
If $0 < \phi < 1$, the **half-life** (time for a shock to decay by 50%) is:

$$
\text{Half-life} = \frac{\ln(0.5)}{\ln(\phi)}
$$

We report half-life in:
- **minutes** (since the data are 1-minute bars)
- optionally **hours** or **days** for readability.

### Interpretation (how to read the number)
- **Short half-life** $\Rightarrow$ deviations correct quickly (strong arbitrage / higher confidence)
- **Long half-life** $\Rightarrow$ deviations persist (market frictions / stress / slower arbitrage)

This answers:
- “Which market/stablecoin reverts faster?”
- “Are deviations more persistent on certain platforms?”


Ensure basis exists (safe re-define):

In [11]:
import numpy as np
import pandas as pd

# --- Ensure Binance basis series exist ---
# Requires: prices_k_clean with columns p_btc_usdt_k, p_btc_usdc_k, p_btc_usd_k
if "basis_usdt_k" not in globals() or "basis_usdc_k" not in globals():
    if "prices_k_clean" not in globals():
        raise NameError("Missing `prices_k_clean`. Re-run your 1.1 load/clean/alignment cells first.")

    p_btc_usdt_k = prices_k_clean["p_btc_usdt_k"]
    p_btc_usdc_k = prices_k_clean["p_btc_usdc_k"]
    p_btc_usd_k  = prices_k_clean["p_btc_usd_k"]

    basis_usdt_k = ((p_btc_usdt_k - p_btc_usd_k) / p_btc_usd_k).rename("basis_usdt_k")
    basis_usdc_k = ((p_btc_usdc_k - p_btc_usd_k) / p_btc_usd_k).rename("basis_usdc_k")

# Put into one aligned DataFrame (drop any missing timestamps just in case)
basis_k = pd.concat([basis_usdt_k, basis_usdc_k], axis=1).dropna()
basis_k.head()


,basis_usdt_k,basis_usdc_k
timestamp,,
2023-01-01 00:09:00+00:00,0.000714,-0.000102
2023-01-01 00:14:00+00:00,0.000454,-0.000091
2023-01-01 00:15:00+00:00,0.000497,0.000005
2023-01-01 00:26:00+00:00,0.000200,-0.000151
2023-01-01 00:33:00+00:00,0.000527,-0.000232


AR(1) OLS fit + half-life function

In [12]:
def ar1_half_life(series: pd.Series) -> dict:
    """
    Fit: y_t = alpha + phi y_{t-1} + eps_t  (OLS)
    Returns phi, alpha, R^2, half-life (minutes) when 0<phi<1, else NaN.
    """
    s = series.dropna().copy()

    y = s.iloc[1:].values
    x_lag = s.iloc[:-1].values

    # OLS with intercept: y = alpha + phi * x_lag
    X = np.column_stack([np.ones_like(x_lag), x_lag])
    beta, *_ = np.linalg.lstsq(X, y, rcond=None)
    alpha, phi = beta[0], beta[1]

    # R^2
    y_hat = X @ beta
    ss_res = np.sum((y - y_hat) ** 2)
    ss_tot = np.sum((y - y.mean()) ** 2)
    r2 = 1 - ss_res / ss_tot if ss_tot > 0 else np.nan

    # Half-life only makes sense in this simple discrete formula if 0<phi<1
    if 0 < phi < 1:
        half_life_min = np.log(0.5) / np.log(phi)
    else:
        half_life_min = np.nan

    return {
        "alpha": alpha,
        "phi": phi,
        "r2": r2,
        "half_life_min": half_life_min,
        "half_life_hr": half_life_min / 60 if np.isfinite(half_life_min) else np.nan,
    }


Compute half-life for USDT basis and USDC basis

In [13]:
res_usdt_k = ar1_half_life(basis_k["basis_usdt_k"])
res_usdc_k = ar1_half_life(basis_k["basis_usdc_k"])

out = pd.DataFrame([res_usdt_k, res_usdc_k], index=["Kraken_USDT_basis", "Kraken_USDC_basis"])
out

,alpha,phi,r2,half_life_min,half_life_hr
Kraken_USDT_basis,-0.000044,0.957552,0.916913,15.980219,0.266337
Kraken_USDC_basis,0.000009,0.996834,0.993679,218.601887,3.643365


Half-life for cross-stablecoin spread (optional for used):

In [14]:
# Ensure spread exists
if "spread_usdt_usdc_k" not in globals():
    spread_usdt_usdc_k = (basis_k["basis_usdt_k"] - basis_k["basis_usdc_k"]).rename("spread_usdt_usdc_k")

res_spread_k = ar1_half_life(spread_usdt_usdc_k)

out2 = pd.DataFrame([res_spread_k], index=["Kraken_USDT_minus_USDC_spread"])
out2


,alpha,phi,r2,half_life_min,half_life_hr
Kraken_USDT_minus_USDC_spread,-0.00001,0.997437,0.994881,270.071641,4.501194


Tiny interpretation helper (prints readable summary):

In [15]:
def pretty_half_life_row(name: str, d: dict):
    phi = d["phi"]
    hlm = d["half_life_min"]
    hlh = d["half_life_hr"]

    print(f"{name}")
    print(f"  phi (AR1): {phi:.6f}")
    print(f"  R^2:       {d['r2']:.4f}")
    if np.isfinite(hlm):
        print(f"  Half-life: {hlm:.2f} minutes  (~{hlh:.2f} hours)")
    else:
        print("  Half-life: NaN (phi not in (0,1) → check persistence/mean-reversion assumption)")
    print()

pretty_half_life_row("Kraken USDT basis", res_usdt_k)
pretty_half_life_row("Kraken USDC basis", res_usdc_k)
pretty_half_life_row("Kraken spread (USDT-USDC)", res_spread_k)


Kraken USDT basis
  phi (AR1): 0.957552
  R^2:       0.9169
  Half-life: 15.98 minutes  (~0.27 hours)

Kraken USDC basis
  phi (AR1): 0.996834
  R^2:       0.9937
  Half-life: 218.60 minutes  (~3.64 hours)

Kraken spread (USDT-USDC)
  phi (AR1): 0.997437
  R^2:       0.9949
  Half-life: 270.07 minutes  (~4.50 hours)



## 2.3 Time Above Threshold (Stress Persistence)

This measure focuses on **how long stablecoin deviations remain elevated**, rather than how large they are at a single moment.

### Stress threshold definition

We define a stress deviation as a basis exceeding a fixed absolute threshold:

$$
|\text{Basis}(t)| > \tau
$$

where the threshold is set to:

$$
\tau = 20 \text{ bps}
$$

(20 basis points corresponds to a 0.20% deviation from parity.)

---

### Measures

For each basis series (USDT, USDC), we compute:

- **Fraction of time above threshold**  
  $$\Pr(|\text{Basis}(t)| > \tau)$$

- **Excursion duration**  
  The number of consecutive minutes during which  
  $$|\text{Basis}(t)| > \tau$$  
  remains true.

We summarize excursion behavior using the **average**, **median**, and **maximum** excursion length.

---

### Interpretation

- Higher time-above-threshold indicates **more persistent stress**
- Longer excursion durations imply **slower arbitrage or weaker confidence**
- Comparing **USDT vs USDC** reveals which stablecoin experiences more prolonged dislocations

---

### Question answered

> **“Do deviations disappear quickly, or do they linger?”**


In [16]:
import numpy as np
import pandas as pd

THRESHOLD_BPS_K = 20
THRESHOLD_K = THRESHOLD_BPS_K / 1e4  # bps -> raw units

def excursion_lengths_k(series: pd.Series, threshold: float) -> pd.Series:
    """
    Returns a Series of excursion lengths (in minutes)
    where |series| > threshold.
    """
    s = series.dropna()
    mask = s.abs() > threshold

    # Identify contiguous blocks
    groups = (mask != mask.shift()).cumsum()

    # Count only True blocks (lengths)
    excursions = mask.groupby(groups).sum()
    return excursions[excursions > 0]

results_k = {}

for label, basis in {
    "USDT (Kraken)": basis_usdt_k,
    "USDC (Kraken)": basis_usdc_k,
}.items():

    b = basis.dropna()
    mask = b.abs() > THRESHOLD_K
    frac_time = mask.mean()

    excursions = excursion_lengths_k(b, THRESHOLD_K)

    if len(excursions) == 0:
        results_k[label] = {
            "fraction_time_above": frac_time,
            "min_excursion_min": np.nan,
            "p25_excursion_min": np.nan,
            "avg_excursion_min": np.nan,
            "median_excursion_min": np.nan,
            "p75_excursion_min": np.nan,
            "max_excursion_min": np.nan,
            "num_excursions": 0,
        }
    else:
        results_k[label] = {
            "fraction_time_above": frac_time,
            "min_excursion_min": excursions.min(),
            "p25_excursion_min": excursions.quantile(0.25),
            "avg_excursion_min": excursions.mean(),
            "median_excursion_min": excursions.median(),
            "p75_excursion_min": excursions.quantile(0.75),
            "max_excursion_min": excursions.max(),
            "num_excursions": len(excursions),
        }

results_23_k = pd.DataFrame(results_k).T
results_23_k["fraction_time_above_pct"] = 100 * results_23_k["fraction_time_above"]

results_23_k

,fraction_time_above,min_excursion_min,p25_excursion_min,avg_excursion_min,median_excursion_min,p75_excursion_min,max_excursion_min,num_excursions,fraction_time_above_pct
USDT (Kraken),0.192748,1.0,1.0,10.396122,2.0,5.0,1780.0,722.0,19.274819
USDC (Kraken),0.086334,1.0,1.0,5.336508,1.0,1.0,2383.0,630.0,8.633352


## 3.1 Stress-Day Conditional Statistics (Calm vs Stress)

This section asks whether stablecoin basis behavior **changes in market stress**.
We split the sample into **calm** vs **stress** periods using BTC market conditions, then compute the same basis metrics in each regime.

---

### Step 1 — Define stress days (based on BTC)

We define 1-minute BTC/USD returns:

$$
r_t = \ln P_{\mathrm{BTC/USD}}(t) - \ln P_{\mathrm{BTC/USD}}(t-1)
$$

Aggregate to daily stress signals (using UTC days):

- **Daily realized volatility**
$$
\sigma_d = \mathrm{StdDev}(r_t \mid t \in d)\times \sqrt{1440}
$$

- **Daily max drawdown** (intraday, from BTC/USD within day)
$$
\mathrm{MDD}_d = \min_{t\in d}\left(\frac{P(t)}{\max_{u\le t,\, u\in d}P(u)} - 1\right)
$$

We label a day as **stress** if it is in the extreme tail of BTC conditions, e.g.:

- volatility in the **top 10%**, OR
- drawdown in the **worst 10%** (most negative)

All other days are labeled **calm**.

---

### Step 2 — Conditional basis statistics

For each basis series (USDT, USDC) we compute, separately in calm vs stress:

- **Mean basis**
- **Volatility of basis** (StdDev)
- **Tail magnitude** (e.g., 95th / 99th percentile of $|\text{Basis}|$)
- **Half-life** (AR(1) mean reversion speed, from Section 2.2)

---

### Interpretation

- If stress-day mean basis becomes more negative or tail magnitudes increase, the stablecoin shows **more severe dislocations under stress**.
- If basis volatility increases under stress, stablecoin pricing becomes **less stable**.
- If half-life increases under stress, deviations **persist longer** (slower arbitrage / weaker confidence).

---

### Questions answered

> “Do USDT and USDC behave similarly in calm periods?”  
> “Which loses the peg more during stress?”


In [17]:
import numpy as np
import pandas as pd

# ---------------------------
# 0) Safety checks (Kraken)
# ---------------------------
required_k = ["prices_k_clean", "basis_usdt_k", "basis_usdc_k"]
missing_k = [x for x in required_k if x not in globals()]
if missing_k:
    raise NameError(f"Missing objects: {missing_k}. Re-run Sections 1.1–2.2 first (Kraken).")

p_btc_k = prices_k_clean["p_btc_usd_k"].dropna().copy()

# ---------------------------
# 1) BTC stress features (daily)
# ---------------------------
# 1-min log returns
r_k = np.log(p_btc_k).diff().dropna()

# Daily realized vol (compare across days)
daily_rv_k = r_k.groupby(r_k.index.floor("D")).std() * np.sqrt(1440)

# Daily max drawdown (intraday within each day)
def daily_max_drawdown_k(price_series: pd.Series) -> float:
    x = price_series.values
    run_max = np.maximum.accumulate(x)
    dd = x / run_max - 1.0
    return float(dd.min())

daily_mdd_k = p_btc_k.groupby(p_btc_k.index.floor("D")).apply(daily_max_drawdown_k)

btc_daily_k = pd.DataFrame({
    "btc_rv_k": daily_rv_k,
    "btc_mdd_k": daily_mdd_k
}).dropna()

# ---------------------------
# 2) Define stress days (quantile rules)
# ---------------------------
Q_VOL_K = 0.90      # top 10% vol
Q_DD_K  = 0.10      # worst 10% drawdown (more negative)

vol_thr_k = btc_daily_k["btc_rv_k"].quantile(Q_VOL_K)
dd_thr_k  = btc_daily_k["btc_mdd_k"].quantile(Q_DD_K)

btc_daily_k["is_stress_k"] = (btc_daily_k["btc_rv_k"] >= vol_thr_k) | (btc_daily_k["btc_mdd_k"] <= dd_thr_k)
btc_daily_k["regime_k"] = np.where(btc_daily_k["is_stress_k"], "stress", "calm")

print("Stress-day thresholds (Kraken):")
print(f"  vol >= {vol_thr_k:.6f} (top {int((1-Q_VOL_K)*100)}%)")
print(f"  mdd <= {dd_thr_k:.6f} (worst {int(Q_DD_K*100)}%)")
print("\nDays per regime:")
print(btc_daily_k["regime_k"].value_counts())

# ---------------------------
# 3) Attach regime label back to minute basis data
# ---------------------------
basis_df_k = pd.concat(
    [
        basis_usdt_k.rename("basis_usdt_k"),
        basis_usdc_k.rename("basis_usdc_k")
    ],
    axis=1
).dropna()

basis_day_k = basis_df_k.index.floor("D")
basis_df_k["regime_k"] = btc_daily_k["regime_k"].reindex(basis_day_k).values
basis_df_k = basis_df_k.dropna(subset=["regime_k"])  # drop if any day label missing

# ---------------------------
# 4) Conditional stats (calm vs stress)
# ---------------------------
def summarize_basis_k(x: pd.Series) -> pd.Series:
    ax = x.abs()
    return pd.Series({
        "mean_bps": x.mean() * 1e4,
        "std_bps": x.std() * 1e4,
        "p95_abs_bps": ax.quantile(0.95) * 1e4,
        "p99_abs_bps": ax.quantile(0.99) * 1e4,
        "max_abs_bps": ax.max() * 1e4,
        "n_minutes": x.shape[0]
    })

out_k = []
for coin_k in ["basis_usdt_k", "basis_usdc_k"]:
    tmp_k = basis_df_k.groupby("regime_k")[coin_k].apply(summarize_basis_k).unstack()
    tmp_k["stablecoin"] = coin_k.replace("basis_", "").replace("_k", "").upper()
    out_k.append(tmp_k.reset_index())

results_31_k = pd.concat(out_k, ignore_index=True).set_index(["stablecoin", "regime_k"])
results_31_k


Stress-day thresholds (Kraken):
  vol >= 0.058991 (top 9%)
  mdd <= -0.050716 (worst 10%)

Days per regime:
regime_k
calm      75
stress    15
Name: count, dtype: int64


mean_bps     std_bps  p95_abs_bps  p99_abs_bps  \
stablecoin regime_k                                                    
USDT       calm      -8.247455   22.123730    54.205065   110.264106   
           stress   -17.053511   19.209341    51.274181    74.109198   
USDC       calm      36.284366  159.382711   288.492458   969.748506   
           stress     7.380489   20.037913    46.231633   102.716068   

                     max_abs_bps  n_minutes  
stablecoin regime_k                          
USDT       calm       166.797100    29919.0  
           stress     109.118868     9023.0  
USDC       calm      1402.747543    29919.0  
           stress     164.082447     9023.0

## 3.2 Asymmetry: Discount vs Premium

This section studies whether stablecoin deviations are **asymmetric**:
do discounts (negative basis) behave differently from premiums (positive basis)?

Instead of looking at overall magnitude, we separate the basis into
**downside (discount)** and **upside (premium)** components.

---

### Definitions

Let $\text{Basis}_e(t)$ denote the stablecoin basis on exchange $e$ at time $t$.

We define:

- **Discounts (downside deviations)**:
\[
\text{Basis}_e(t) < 0
\]

- **Premiums (upside deviations)**:
\[
\text{Basis}_e(t) > 0
\]

We compute conditional moments separately:

- Mean discount magnitude:
\[
\mathbb{E}\left[\,|\text{Basis}_e(t)| \mid \text{Basis}_e(t) < 0\,\right]
\]

- Mean premium magnitude:
\[
\mathbb{E}\left[\,\text{Basis}_e(t) \mid \text{Basis}_e(t) > 0\,\right]
\]

---

### Interpretation

- **Discounts** typically reflect:
  - confidence shocks,
  - redemption or counterparty concerns,
  - stress-driven selling of stablecoins.

- **Premiums** often reflect:
  - liquidity demand,
  - funding frictions,
  - demand for fast settlement or exchange-specific constraints.

Asymmetry between discounts and premiums is informative for
**regulatory confidence** and **market stress diagnostics**.

---

### Questions Answered

- Do stablecoins experience **larger downside shocks than upside moves**?
- Is loss of confidence sharper than liquidity-driven demand?
- Do USDT and USDC behave asymmetrically during stress periods?


In [18]:
import numpy as np
import pandas as pd

# ---------------------------
# 3.2 Asymmetry (Kraken)
# ---------------------------
# Safety checks
required = ["basis_usdt_k", "basis_usdc_k"]
missing = [x for x in required if x not in globals()]
if missing:
    raise NameError(f"Missing objects: {missing}. Re-run Sections 1.1–2.2 first.")

# Helper function to compute asymmetry stats
def asymmetry_stats(basis: pd.Series, label: str) -> pd.Series:
    basis = basis.dropna()
    discounts = basis[basis < 0]
    premiums  = basis[basis > 0]

    return pd.Series({
        "obs_total": len(basis),
        "share_discounts": (basis < 0).mean(),
        "share_premiums":  (basis > 0).mean(),
        "mean_discount_abs": discounts.abs().mean(),          # level (not bps yet)
        "mean_premium": premiums.mean(),                      # level
        "p95_discount_abs": discounts.abs().quantile(0.95),   # level
        "p95_premium": premiums.quantile(0.95),               # level
    }, name=label)

# Compute asymmetry for Kraken
asym_usdt_k = asymmetry_stats(basis_usdt_k, "USDT (Kraken)")
asym_usdc_k = asymmetry_stats(basis_usdc_k, "USDC (Kraken)")

asymmetry_table_k = pd.concat([asym_usdt_k, asym_usdc_k], axis=1)
display(asymmetry_table_k)

,USDT (Kraken),USDC (Kraken)
obs_total,38942.000000,38942.000000
share_discounts,0.682682,0.400570
share_premiums,0.313620,0.598634
mean_discount_abs,0.001656,0.000401
mean_premium,0.000325,0.005211
p95_discount_abs,0.006982,0.001145
p95_premium,0.000878,0.040343


Same results in basis points:

In [19]:
# Same results in basis points:
asymmetry_bps_k = asymmetry_table_k.copy()

# rows that are "levels" -> convert to bps
rows_to_bps = ["mean_discount_abs", "mean_premium", "p95_discount_abs", "p95_premium"]
asymmetry_bps_k.loc[rows_to_bps, :] = asymmetry_bps_k.loc[rows_to_bps, :] * 1e4

display(asymmetry_bps_k)


,USDT (Kraken),USDC (Kraken)
obs_total,38942.000000,38942.000000
share_discounts,0.682682,0.400570
share_premiums,0.313620,0.598634
mean_discount_abs,16.562499,4.007655
mean_premium,3.249371,52.106276
p95_discount_abs,69.824854,11.452343
p95_premium,8.779722,403.433542
